In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.multiprocessing

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transform
transform = transforms.Compose(
    [transforms.Resize(32), transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

# Dataset
trainset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2
)

testset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=False, num_workers=2
)


# Model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((4, 4)),
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
        )

    def forward(self, x):
        return self.net(x)


# Loss and Optimizer
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
EPOCHS = 30

if __name__ == '__main__':
    torch.multiprocessing.freeze_support()

    for epoch in range(EPOCHS):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {running_loss/len(trainloader):.4f}")

    print("Finished Training")

    # Testing
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy on the 10000 test images: {100 * correct / total:.2f}%")


Epoch 1/30 - Loss: 1.3918
Epoch 2/30 - Loss: 0.9711
Epoch 3/30 - Loss: 0.7830
Epoch 4/30 - Loss: 0.6590
Epoch 5/30 - Loss: 0.5599
Epoch 6/30 - Loss: 0.4695
Epoch 7/30 - Loss: 0.3897
Epoch 8/30 - Loss: 0.3185
Epoch 9/30 - Loss: 0.2469
Epoch 10/30 - Loss: 0.1960
Epoch 11/30 - Loss: 0.1528
Epoch 12/30 - Loss: 0.1283
Epoch 13/30 - Loss: 0.1026
Epoch 14/30 - Loss: 0.0857
Epoch 15/30 - Loss: 0.0891
Epoch 16/30 - Loss: 0.0666
Epoch 17/30 - Loss: 0.0784
Epoch 18/30 - Loss: 0.0634
Epoch 19/30 - Loss: 0.0626
Epoch 20/30 - Loss: 0.0615
Epoch 21/30 - Loss: 0.0590
Epoch 22/30 - Loss: 0.0616
Epoch 23/30 - Loss: 0.0491
Epoch 24/30 - Loss: 0.0500
Epoch 25/30 - Loss: 0.0533
Epoch 26/30 - Loss: 0.0499
Epoch 27/30 - Loss: 0.0474
Epoch 28/30 - Loss: 0.0466
Epoch 29/30 - Loss: 0.0455
Epoch 30/30 - Loss: 0.0435
Finished Training
Accuracy on the 10000 test images: 75.90%
